# 4장 Spatial Data Import and Export

- Open Source Geospatial Foundation( OSGF )에서 오픈소스 모아짐.
- 이런 오픈소스들은 Geospatial Data Abstraction Library( GDAL )과 PROJ.4을 많이 사용함.


- 이장의 학습목표
    - 건고하고 이식성이 좋은 방법으로 기본좌표계를 표현 방법을 고려함.
    - 가장 대중적인 포맷을 사용해서 공간데이터를 R로 읽고 쓰는 방법을 보여줌.
    - GRASS GIS와의 인터페이스를 자세히 다룸.
    - visualisation을 위한 데이터 export을 함.
  
  
- rgdal 패키지를 로딩하는 것으로부터 시작함.~~~

In [1]:
library(rgdal)

Loading required package: sp
rgdal: version: 1.1-8, (SVN revision 616)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.0.1, released 2015/09/15
 Path to GDAL shared files: C:/Anaconda2/R/library/rgdal/gdal
 GDAL does not use iconv for recoding strings.
 Loaded PROJ.4 runtime: Rel. 4.9.1, 04 March 2015, [PJ_VERSION: 491]
 Path to PROJ.4 shared files: C:/Anaconda2/R/library/rgdal/proj
 Linking to sp version: 1.2-3 


## 4.1 Coordinate Reference Systems

- 지구는 울퉁불퉁한 타원이고, 각각의 국가마다 기준과 측정 단위( km, feet )가 다르기 때문에 매우 다양함.
- 북해분지( the North Sea basin )의 해안선이 호환되지 않고 불충분한 CRS로 정의되어서  European Petroleum Survey Group( EPSG )에서 1986년부터 geodetic parameter data set을 수집하기 시작함.

### 4.1.1 Using the EPSG List

- EPSG list는 rgdal 패키지에 제공됨.
- 대부분의 CRS 데이터는 이 책에서 사용하는 PROJ.4 style로 변환되어짐.
- ED50( the European Datum 1950 )안의 차트좌표를 WGS84 datum안에서의 차트좌표로 변환이 필요함.

In [2]:
EPSG <- make_EPSG()
EPSG[grep("^# ED50$", EPSG$note), ]

,code,note,prj4
159,4230,# ED50,"+proj=longlat +ellps=intl +towgs84=-87,-98,-121,0,0,0,0 +no_defs"


- EPSG code가 첫번째 컬럼에 있고,  PROJ.4 specification은 세번째 컬럼에 있음.

### 4.1.2 PROJ.4 CRS Specification

- PROJ.4 library는 기준좌표계를 "태그=값" 형식으로 표현.
- CRS class는 기존좌표에 대한 정보가 없으면( NA ) unknown CRS으로 인식하고 지리적인 좌표을 포함한 CRS는 longlat 문자열을 포함함.
- 태그는 +로 시작하고 = 부호로 구분으로 태그와 값을 나눔.

In [3]:
CRS("+init=epsg:4230")

CRS arguments:
 +init=epsg:4230 +proj=longlat +ellps=intl +towgs84=-87,-98,-121,0,0,0,0
+no_defs 

- +proj 태그는 지리적인 좌표로 longlat( 위경도 )값을 갖고,  +ellps 태그는 International Ellipsoid of 1909을 나타내는 intl 라는 값을 갖음.

In [4]:
ED50 <- CRS("+init=epsg:4230 +towgs84=-87,-96,-120,0,0,0,0")
ED50

CRS arguments:
 +init=epsg:4230 +towgs84=-87,-96,-120,0,0,0,0 +proj=longlat +ellps=intl
+no_defs 

- 자료변환은 3차원에서 다른 특별한 타원형(ellipsoids) 간에서 좌표가 이동됨.

### 4.1.3 Projection and Transformation



In [5]:
IJ.east <- as(char2dms("4d31'00\"E"), "numeric")
IJ.north <- as(char2dms("52d28'00\"N"), "numeric")
IJ.ED50 <- SpatialPoints(cbind(x = IJ.east, y = IJ.north),  ED50)
res <- spTransform(IJ.ED50, CRS("+proj=longlat +datum=WGS84"))
x <- as(dd2dms(coordinates(res)[1]), "character")
y <- as(dd2dms(coordinates(res)[2], TRUE), "character")
cat(x, y, "\n")

4d30'55.294"E 52d27'57.195"N 


In [7]:
spDistsN1(coordinates(IJ.ED50), coordinates(res), longlat = TRUE) *  1000

[1] 124.0994

In [ ]:
library(maptools)

In [9]:
gzAzimuth(coordinates(IJ.ED50), coordinates(res))

x 
-134.3674

- 위의 코드에서 ED50 datum와 WGS84 datum 사이의 124m 이동됨.
- spTransform()함수는 Spatial* 객체을 인자도 받아서, 타켓 CRS로 기준좌표계를 변환해서 리턴함.
- spDistsN1()함수와 gzAzimuth()함수는 ED50와  WGS84에서 포인트간의 차이를 계산하는 함수임.
- spDistsN1()함수는 두점 사이의 거리 측정
- gzAzimuth()함수는 두점 사이의 sphere(구쳬)에서의 방위각(azimuths)을 측정

In [11]:
EPSG[grep("Atlas", EPSG$note), 1:2]

,code,note
639,2163,# US National Atlas Equal Area
2341,3978,# NAD83 / Canada Atlas Lambert
2342,3979,# NAD83(CSRS) / Canada Atlas Lambert


In [14]:
CRS("+init=epsg:2163")

CRS arguments:
 +init=epsg:2163 +proj=laea +lat_0=45 +lon_0=-100 +x_0=0 +y_0=0
+a=6370997 +b=6370997 +units=m +no_defs 

- US National Atlas는 특별한 CRS로 선택되어짐.
- laea은 CRS in PROJ.4 로 표현하는 다른 값임.

In [16]:
proj <- projInfo("proj")
proj[proj$name == "laea", ]

,name,description
53,laea,Lambert Azimuthal Equal Area


In [18]:
ellps <- projInfo("ellps")
ellps[grep("a=6370997", ellps$major), ]

,name,major,ell,description
43,sphere,a=6370997.0,b=6370997.0,Normal Sphere (r=6370997)


- Lambert Azimuthal Equal Area projection은 100◦west and 45◦ north을 중심으로 복잡한 타원형보다는 구형을 사용함.

### 4.1.4 Degrees, Minutes, and Seconds

- 위경도 좌표 는 degrees(도), minutes(분), decimal seconds (초)로 표현
- char2dms()함수를 사용해서 위경도 좌표 문자열을 DMS 객체로 변환함.

In [19]:
IJ.dms.E <- "4d31'00\"E"
IJ.dms.N <- "52d28'00\"N"

In [21]:
IJ_east <- char2dms(IJ.dms.E)
IJ_north <- char2dms(IJ.dms.N)
IJ_east

[1] 4d31'E

In [23]:
IJ_north

[1] 52d28'N

In [24]:
getSlots("DMS")

WS       deg       min       sec        NS 
"logical" "numeric" "numeric" "numeric" "logical"

- DMS class는 지리적인 좌표 표현을 저장함.
- as() 함수를 사용해서 십진수 degrees(도)로 값을 변환함.

In [25]:
c(as(IJ_east, "numeric"), as(IJ_north, "numeric"))

[1]  4.516667 52.466667

## 4.2 Vector File Formats

- Spatial vector data는 점, 선, 다각형, sp class임.
- Vector formats은 R 이외에서 import가 될 수 있는 포맷임.
- sp vector classes은 경계선을 체크하지 않고 저장되는 각각의 다각형이라는 의미에서 단순함.

### 4.2.1 Using OGR Drivers in rgdal

- rgdal패키지안의 Geospatial Data Abstraction Library의 OGR vector 함수를 사용해서 spatial vector data을 읽어보자.
- OGR은 직접적으로 기준좌표계을 핸들링을 지원해서 규격화된 imported data을 읽어올 수 있음.
- ogrDrivers()함수는 출력파일을 생성할 수 있는 driver가 무엇인지 리스트를 보여줌.
- readOGR()함수는 data source name (dsn)와 layer (layer)를 최소한 2개의 인자가 필요. 
- http://ogr.maptools.org/ogr_formats.html에서 import되는 포맷을 볼수 있음.


- Scottish lip cancer 데이터를 직접다운받음.
- 3개의 파일은  데이터 수집시점에 스코틀랜드 지역 구분에 대한 정보임.
- scotland.dat 파일은 원천 연구 데이터임.

ogrDrivers()

In [1]:
download.file("http://web1.sph.emory.edu/users/lwaller/book/ch9/scot.shp", "scot.shp", mode="wb")
#download.file("http://www.sph.emory.edu/~lwaller/book/ch9/scot.shp", "scot.shp", mode="wb")
download.file("http://web1.sph.emory.edu/users/lwaller/book/ch9/scot.dbf", "scot.dbf", mode="wb")
download.file("http://web1.sph.emory.edu/users/lwaller/book/ch9/scot.shx", "scot.shx", mode="wb")
download.file("http://web1.sph.emory.edu/users/lwaller/book/ch2/scotland.dat", "scotland.dat", mode="w")

In [7]:
scot_LL <- readOGR(".", "scot")
proj4string(scot_LL) <- CRS("+proj=longlat +ellps=WGS84")
scot_LL$ID

OGR data source with driver: ESRI Shapefile 
Source: ".", layer: "scot"
with 56 features
It has 2 fields


[1] 12 13 19  2 17 16 21 50 15 25 26 29 43 39 40 52 42 51 34 54 36 46 41 53 49
[26] 38 44 30 45 48 47 35 28  4 20 33 31 24 55 18 56 14 32 27 10 22  6  8  9  3
[51]  5 11  1  7 23 37

In [8]:
scot_dat <- read.table("scotland.dat", skip = 1)
names(scot_dat) <- c("District", "Observed", "Expected", 
                     "PcAFF", "Latitude", "Longitude")
scot_dat$District

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56

In [7]:
head( scot_dat )

,District,Observed,Expected,PcAFF,Latitude,Longitude
1,1,9,1.4,16,57.29,5.5
2,2,39,8.7,16,57.56,2.36
3,3,11,3,10,58.44,3.9
4,4,9,2.5,24,55.76,2.4
5,5,15,4.3,10,57.71,5.09
6,6,8,2.4,24,59.13,3.25


In [9]:
library(maptools)
scot_dat1 <- scot_dat[match(scot_LL$ID, scot_dat$District), ]
row.names(scot_dat1) <- sapply(slot(scot_LL, "polygons"), 
                               function(x) slot(x, "ID"))
scot_LLa <- spCbind(scot_LL, scot_dat1)
all.equal(scot_LLa$ID, scot_LLa$District)
names(scot_LLa)

[1] TRUE

[1] "NAME"      "ID"        "District"  "Observed"  "Expected"  "PcAFF"    
[7] "Latitude"  "Longitude"

- 이 데이터셋을 가지고 11장에서 아래와 같이 그리는 방법을 설명함.

![](chapter04_01.png)

In [ ]:
install.packages('spdep', repos="http://cran.nexr.com/"  )

In [ ]:
install.packages('DCluster', repos="http://cran.nexr.com/"  )

In [17]:
library(spdep)
O <- scot_LLa$Observed
E <- scot_LLa$Expected
scot_LLa$SMR <- probmap(O, E)$relRisk/100
library(DCluster)
scot_LLa$smth <- empbaysmooth(O, E)$smthrr

Loading required package: boot
Loading required package: MASS


In [18]:
scot_BNG <- spTransform(scot_LLa, CRS("+init=epsg:27700"))

- Google Earth에 표시할 수 있도록 Keyhole Markup Language(KML)로 데이터 export할 수 있음.

In [ ]:
writeOGR(scot_LLa["ID"], dsn = "scot_district.kml",layer = "borders", driver = "KML")
llCRS <- CRS("+proj=longlat ellps=WGS84")
scot_SP_LL <- SpatialPointsDataFrame(coordinates(scot_LLa), 
                                     proj4string = llCRS, 
                                     data = as(scot_LLa, "data.frame")[c("NAME","Observed", "Expected", "SMR", "smth")])
writeOGR(scot_SP_LL, dsn = "scot_rates.kml", 
         layer = "rates", driver = "KML")

![](chapter04_02.png)

## 4.3 Raster File Formats

- raster File Format은 Red-Green-Blue (RGB)값을 갖음.
- rimage ,  biOps, EBImage(Bioconductor) 패키지가 raster image을 import와 export 기능을 제공.
- spatial raster data는 기준좌표에 대한 핸들링이 필요함.
- 그래서, 위의 패키지는 직접 지원하지 않음.

### 4.3.1 Using GDAL Drivers in rgdal

- rgdal패키지의 readGDAL()함수는 다양한 driver 지원함.

In [ ]:
![](70042108.tif)

In [10]:
auck_el1 <- readGDAL("70042108.tif")

70042108.tif has GDAL driver GTiff 
and has 1200 rows and 1320 columns


In [11]:
summary(auck_el1)

Object of class SpatialGridDataFrame
Coordinates:
    min   max
x 174.2 175.3
y -37.5 -36.5
Is projected: FALSE 
proj4string :
[+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0]
Grid attributes:
  cellcentre.offset     cellsize cells.dim
x         174.20042 0.0008333333      1320
y         -37.49958 0.0008333333      1200
Data attributes:
     band1           
 Min.   :-3.403e+38  
 1st Qu.: 0.000e+00  
 Median : 1.000e+00  
 Mean   :-1.869e+34  
 3rd Qu.: 5.300e+01  
 Max.   : 6.860e+02  

- readGDAL()함수는 GDAL과 R을 결합시켜주는 강력한 기능을 갖음.
- 단지 읽기만 필요하면, GDALDriver 클래스를 사용할 수 있음

In [12]:
x <- GDAL.open("70042108.tif")
xx <- getDriver(x)
xx

An object of class "GDALDriver"
Slot "handle":
<pointer: 0x0000000012a7eea0>


In [13]:
getDriverLongName(xx)

[1] "GeoTIFF"

In [14]:
x

An object of class "GDALReadOnlyDataset"
Slot "handle":
<pointer: 0x000000001215fe30>


In [7]:
dim(x)

[1] 1200 1320

In [9]:
GDAL.close(x)

- GDALinfo함수를 이용해서 raster file에 대한 정보를 쉽게 볼 수 있음

In [15]:
GDALinfo("70042108.tif")

Warning message:
: statistics not supported by this driver

rows        1200 
columns     1320 
bands       1 
lower left origin.x        174.2 
lower left origin.y        -37.5 
res.x       0.0008333333 
res.y       0.0008333333 
ysign       -1 
oblique.x   0 
oblique.y   0 
driver      GTiff 
projection  +proj=longlat +datum=WGS84 +no_defs 
file        70042108.tif 
apparent band summary:
   GDType hasNoDataValue NoDataValue blockSize1 blockSize2
1 Float32          FALSE           0          1       1320
apparent band statistics:
         Bmin       Bmax Bmean Bsd
1 -4294967295 4294967295    NA  NA
Metadata:
AREA_OR_POINT=Area 
TIFFTAG_RESOLUTIONUNIT=1 (unitless) 
TIFFTAG_SOFTWARE=IMAGINE TIFF Support
Copyright 1991 - 1999 by ERDAS, Inc. All Rights Reserved
@(#)$RCSfile: etif.c $ $Revision: 1.11 $ $Date$ 
TIFFTAG_XRESOLUTION=1 
TIFFTAG_YRESOLUTION=1 

- Meuse(프랑스의 지명, 로테) grid 데이터셋로 GDAL을 사용해서 쓰기를 어떻게 하는지 보여줌.
- writeGDAL()함수는 driver을 직접사용해서 함.
- GeoTiff 파일로 출력하기 위해서 zinc(아연) ppm값을 로그취해서 거리 가중 보간값의 역값으로 출력함.

In [31]:
library(sp)
data(meuse.grid)
coordinates(meuse.grid) <- c("x", "y")
gridded(meuse.grid) <- TRUE
proj4string(meuse.grid) <- CRS("+init=epsg:28992")
data(meuse)
coordinates(meuse) <- c("x", "y")
proj4string(meuse) <- CRS(proj4string(meuse.grid))

In [32]:
library(gstat)
log_zinc <- gstat::idw(log(zinc)~1, meuse, meuse.grid)["var1.pred"]

[inverse distance weighted interpolation]


In [33]:
proj4string(log_zinc) <- CRS(proj4string(meuse.grid))
summary(log_zinc)

Object of class SpatialPixelsDataFrame
Coordinates:
     min    max
x 178440 181560
y 329600 333760
Is projected: TRUE 
proj4string :
[+init=epsg:28992 +proj=sterea +lat_0=52.15616055555555
+lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000
+ellps=bessel
+towgs84=565.4171,50.3319,465.5524,-0.398957388243134,0.343987817378283,-1.87740163998045,4.0725
+units=m +no_defs]
Number of points: 3103
Grid attributes:
  cellcentre.offset cellsize cells.dim
x            178460       40        78
y            329620       40       104
Data attributes:
   var1.pred    
 Min.   :4.791  
 1st Qu.:5.484  
 Median :5.694  
 Mean   :5.777  
 3rd Qu.:6.041  
 Max.   :7.482  

In [34]:
writeGDAL(log_zinc, fname = "log_zinc.tif", 
          driver = "GTiff", type = "Float32", options = "INTERLEAVE=PIXEL")
GDALinfo("log_zinc.tif")

Warning message:
: statistics not supported by this driver

rows        104 
columns     78 
bands       1 
lower left origin.x        178440 
lower left origin.y        329600 
res.x       40 
res.y       40 
ysign       -1 
oblique.x   0 
oblique.y   0 
driver      GTiff 
projection  +proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889
+k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel
+towgs84=565.4171,50.3319,465.5524,-0.398957388243134,0.343987817378283,-1.87740163998045,4.0725
+units=m +no_defs 
file        log_zinc.tif 
apparent band summary:
   GDType hasNoDataValue NoDataValue blockSize1 blockSize2
1 Float32          FALSE           0         26         78
apparent band statistics:
         Bmin       Bmax Bmean Bsd
1 -4294967295 4294967295    NA  NA
Metadata:
AREA_OR_POINT=Area 

![](log_zinc.tif)

### 4.3.2 Writing a Google Earth™ Image Overlay

- 좀더 유용하게 raster 데이터를 저장할 수 있음.
- Google Earth사용하는 PNG 이미지에 위의 보정된 log zinc ppm값을 색상값으로 export 함.

In [ ]:
library(maptools)
grd <- as(meuse.grid, "SpatialPolygons")
proj4string(grd) <- CRS(proj4string(meuse))
grd.union <- unionSpatialPolygons(grd, rep("x", length(slot(grd, "polygons"))))
ll <- CRS("+proj=longlat +datum=WGS84")
grd.union.ll <- spTransform(grd.union, ll)

In [ ]:
llGRD <- GE_SpatialGrid(grd.union.ll)
llGRD_in <- overlay(llGRD$SG, grd.union.ll)
llSGDF <- SpatialGridDataFrame(grid = slot(llGRD$SG,"grid"), 
                               proj4string = CRS(proj4string(llGRD$SG)),
                               data = data.frame(in0 = llGRD_in))
 llSPix <- as(llSGDF, "SpatialPixelsDataFrame")

In [ ]:
meuse_ll <- spTransform(meuse, CRS("+proj=longlat +datum=WGS84"))
llSPix$pred <- idw(log(zinc) ~ 1, meuse_ll, llSPix)$var1.pred

png(file = "zinc_IDW.png", width = llGRD$width,height = llGRD$height, bg = "transparent")
par(mar = c(0, 0, 0, 0), xaxs = "i", yaxs = "i")
image(llSPix, "pred", col = bpy.colors(20))
dev.off()
kmlOverlay(llGRD, "zinc_IDW.kml", "zinc_IDW.png")

![](chapter04_03.png)

## 4.4 Grass

- Grass은 미육군 건설기술 연구소에서 Geographic Resources Analysis Support System으로써 개발되어고 오픈소스임. 
- https://grass.osgeo.org/

In [2]:
install.packages( 'spgrass6',   repos="http://cran.nexr.com/"  ) 

also installing the dependency 'XML'



package 'XML' successfully unpacked and MD5 sums checked
package 'spgrass6' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Y.G JI\AppData\Local\Temp\RtmpgfX4Qi\downloaded_packages


In [1]:
library(spgrass6)

Loading required package: sp
Loading required package: XML
GRASS GIS interface loaded with GRASS version: (GRASS not running)


In [2]:
system("g.version", intern = TRUE)

ERROR: Error in system("g.version", intern = TRUE): 'g.version' not found


In [3]:
library(spgrass6)
gmeta6()

ERROR: Error in parseGRASS(cmd, legacyExec = legacyExec): g.region not found
